<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EC%A2%85%ED%95%A9%EC%9D%98%EA%B2%AC_%EC%83%9D%EA%B8%B0%EB%B6%80%EC%A0%84%EC%B2%98%EB%A6%AC_2024_%EC%9D%80%ED%8F%89%EA%B3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#모든반의 자료를 읽고 전처리하여 하나의 엑셀로 저장

import os
import pandas as pd

# Path where all files are located
path =  '/content/drive/MyDrive/python/eunpyeong/srecord/jonghap/3rdyear/'

# Output path where to save the combined file
output_path = '/content/drive/MyDrive/python/eunpyeong/result_srecord/jonghap/'
output_filename = '3학년종합의견_전체_october.xlsx'

# List all files in the directory
files = [f for f in os.listdir(path) if f.endswith('.xlsx')]

# Initialize an empty DataFrame to combine all the results
combined_df = pd.DataFrame()

for file in files:
    # Load each Excel file
    df = pd.read_excel(os.path.join(path, file))

    # Extract year and class information from the second row
    year_number = df.iloc[1, 0].split()[0][0]  # Extracts '3' from '3학년'
    class_number = df.iloc[1, 0].split()[1][0]  # Extracts '1' from '3학년 1반'

    # Set the row containing '번 호', '성 명', '학 년' as the header
    df.columns = df.iloc[2]
    df_cleaned = df.drop([0, 1, 2])

    # Filter relevant columns
    df_cleaned = df_cleaned[['번 호', '성  명', '행 동 특 성   및   종 합 의 견 ']]

    # Convert '번 호' to numeric for comparison, coercing errors to NaN
    df_cleaned['번 호'] = pd.to_numeric(df_cleaned['번 호'], errors='coerce')

    # Remove rows where the values in '번 호', '성  명', or '학 년' columns are equal to their respective column names
    df_cleaned = df_cleaned[(df_cleaned['번 호'] != '번 호') & (df_cleaned['성  명'] != '성  명')]

    # Rename columns
    df_cleaned.rename(columns={'번 호': '번호', '성  명':'성명', '행 동 특 성   및   종 합 의 견 ': '행동특성및종합의견'}, inplace=True)

    # Fill missing values for '번호' and '성명'
    df_cleaned['번호'] = df_cleaned['번호'].ffill()
    df_cleaned['성명'] = df_cleaned['성명'].ffill()

    # Remove rows where '행동특성및종합의견' has missing values
    df_cleaned = df_cleaned.dropna(subset=['행동특성및종합의견'])

    # Sort by '번호'
    df_cleaned['번호'] = df_cleaned['번호'].astype(int)
    df_cleaned = df_cleaned.sort_values(by='번호')

    # Add year and class information
    df_cleaned['학년'] = year_number
    df_cleaned['반'] = class_number


    # Keep only the required columns in the specified order
    df_cleaned = df_cleaned[['학년', '반', '번호', '성명', '행동특성및종합의견']]

    # Append to the combined dataframe
    combined_df = pd.concat([combined_df, df_cleaned], ignore_index=True)

# Save the combined result to the output path
output_file = os.path.join(output_path, output_filename)
combined_df.to_excel(output_file, index=False)


#경고 차단
pd.set_option('future.no_silent_downcasting', True)


print(f"'{output_filename}' 파일로 전처리된 데이터가 저장되었습니다.")

'3학년종합의견_전체_october.xlsx' 파일로 전처리된 데이터가 저장되었습니다.


In [ ]:
#하나의 반을 개별적으로 부르기 - 투박한 엑셀결과

import pandas as pd


##  데이터 부르기
path = '/content/drive/MyDrive/python/eunpyeong/srecord/jonghap/3rdyear/'
df = pd.read_excel(f'{path}srecord_301_jonghap.xlsx')

#인덱스행 결정
df.columns  = df.iloc[2]

#불필요한 1~3행을 삭제: 여기서 1은 초기 인덱스행을 의미함
df_cleaned = df.drop([0, 1, 2])


#필요한 열만 취함
df_cleaned = df_cleaned[['번 호', '성  명', '학 년', '행 동 특 성   및   종 합 의 견 ']]


# Convert '번 호' to numeric for comparison, coercing errors to NaN
df_cleaned['번 호'] = pd.to_numeric(df_cleaned['번 호'], errors='coerce')


# Remove rows where the values in '번 호', '성  명', or '학 년' columns are equal to their respective column names
df_cleaned = df_cleaned[(df_cleaned['번 호'] != '번 호') &
                        (df_cleaned['성  명'] != '성  명') &
                        (df_cleaned['학 년'] != '학 년')]



# Rename '번 호' to '번호' and '성 명' to '성명'
df_cleaned.rename(columns={'번 호': '번호', '성  명':'성명', '학 년': '학년', '행 동 특 성   및   종 합 의 견 ': '행동특성및종합의견'}, inplace=True)


#결측값 제외
df_cleaned['번호'] = df_cleaned['번호'].ffill()
df_cleaned['성명'] = df_cleaned['성명'].ffill()


# Remove rows where '행동특성및종합의견' has missing values
df_cleaned = df_cleaned.dropna(subset=['행동특성및종합의견'])


#필요한 열만 취함
df_cleaned = df_cleaned[['번호', '성명', '행동특성및종합의견']]


# Group by '성명' and aggregate '특기사항' by joining the values
df_cleaned = df_cleaned.groupby('성명').agg({'번호': 'first',  # Keep the first '번호' value
            '행동특성및종합의견': ' '.join  # Concatenate all '특기사항' values
        }).reset_index()


# Change '번호' values to integers
df_cleaned['번호'] = df_cleaned['번호'].astype(int)

# Sort the DataFrame by '번호' in ascending order
df_cleaned = df_cleaned.sort_values(by='번호')

# Keep only the required columns in the specified order
df_cleaned = df_cleaned[['번호', '성명', '행동특성및종합의견']]


df_cleaned.head(20)